In [3]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



## Определение пустых report_date в _ch подтягивание из источников

In [ ]:
import clickhouse_connect
import os
import datetime
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta

PASSWORD = os.getenv("PASSWORD")
class ClickHouse_client:
    def __init__(self):
        self.client = clickhouse_connect.get_client(host='rc1a-eflrt968scbm232q.mdb.yandexcloud.net', 
                                    port=8443, 
                                    username='Boris', 
                                    password=PASSWORD, 
                                    secure=True,
                                    verify=False,
                                    database='db1')

    def query_run (self, query_text):
        query_text = query_text
        result = self.client.query(query_text)
        self.df = pd.DataFrame(result.result_rows, columns=result.column_names)
        # self.df = pl.DataFrame(result.result_rows, schema=result.column_names)
        # self.df =  pl.from_pandas(self.df)
        # display(self.df)
        return self.df

ch = ClickHouse_client()

def insert_new_date(table, special=False):
    query_text = f"""
        SELECT source.report_date AS report_date
        FROM (SELECT DISTINCT report_date FROM `{table}_mv_df`) AS source
        LEFT ANTI JOIN
        (
            SELECT DISTINCT report_date
            FROM `{table}_ch`
        ) ch USING (report_date)
        ORDER BY source.report_date
    """

    df = ch.query_run(query_text)      # df: pandas.DataFrame
    missing = df['report_date'].tolist() if not df.empty else []
    dates_pd = pd.DataFrame({
            'report_date': [d.strftime('%Y-%m-%d') for d in missing],
            'date_key_1': [d.strftime('%%/year=%-Y/month=%m/%d%%') for d in missing], 
            'date_key_2': [d.strftime('%%/year=%-Y/month=%-m/%d%%') for d in missing], 
            'date_key_3': [d.strftime('%%/year=%-Y/month=%-m/%-d%%') for d in missing],
            'date_key_4': [d.strftime('%%/year=%-Y/month=%m/%-d%%') for d in missing],  
            })
    dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
    
    if special:
        for day_index in dates_pd.index:
            report_date = dates_pd.loc[day_index,['report_date']].values[0]
            date_key_1 = dates_pd.loc[day_index,['date_key_1']].values[0]
            date_key_2 = dates_pd.loc[day_index,['date_key_2']].values[0]
            date_key_3 = dates_pd.loc[day_index,['date_key_3']].values[0]
            date_key_4 = dates_pd.loc[day_index,['date_key_4']].values[0]
            query_text = open(f'{table}_ch.txt','r').read().format(report_date,date_key_1,date_key_2,date_key_3,date_key_4)
            # приведём к литералу ClickHouse (подберите нужный тип)
            ch.query_run(query_text)
    else:
        for day_index in dates_pd.index:
            report_date = dates_pd.loc[day_index,['report_date']].values[0]
            date_key_1 = dates_pd.loc[day_index,['date_key_1']].values[0]
            date_key_2 = dates_pd.loc[day_index,['date_key_2']].values[0]
            date_key_3 = dates_pd.loc[day_index,['date_key_3']].values[0]
            date_key_4 = dates_pd.loc[day_index,['date_key_4']].values[0]
            query_text = open(f'regular_query.txt','r').read().format(report_date,date_key_1,date_key_2,date_key_3,date_key_4,table)
            # приведём к литералу ClickHouse (подберите нужный тип)
            ch.query_run(query_text)


tables_list_spacial = ['citizens_st_mobile','companies_st_partner','intercoms_st_asgard']

tables_list = ch.query_run('SHOW TABLES')
tables_list = tables_list['name'].to_list()
filtered_tables = [
    t for t in tables_list
    if ('_st_' in t 
            or 'hex_metrics_parquet' in t
            or 'metrics_all_intercoms_asgard' in t
            or 'metrics_asgard' in t
        ) 
        and not (t.endswith('_ch') 
                or t.endswith('_mv') 
                or t.endswith('_df') 
                or t.startswith('t_') 
                or t.endswith('_data')
                or t.endswith('_ch_2')
                or t.endswith('_mv_2'))
        and t not in tables_list_spacial
]

def handler(event, context):
    for table in tables_list_spacial:
        insert_new_date(table,special=True)

    for table in filtered_tables:
        insert_new_date(table,special=False)
    




## Создание MV с подсчетом значений по дням для таблиц

In [ ]:
def unwrap_type(t: str) -> str:
    """
    Убираем внешние обёртки типа: Nullable(...), LowCardinality(...),
    SimpleAggregateFunction(..., T) и т.п., чтобы получить базовый тип.
    """
    tl = t.strip()
    changed = True
    while changed:
        changed = False
        for prefix in ('Nullable(', 'LowCardinality(', 'SimpleAggregateFunction('):
            if tl.startswith(prefix):
                # SimpleAggregateFunction(ARG,..., T) -> берём последнюю запятую как разделитель T
                if prefix == 'SimpleAggregateFunction(':
                    inner = tl[len(prefix):-1]  # без последней ")"
                    # тип — после последней запятой
                    if ',' in inner:
                        tl = inner.split(',')[-1].strip()
                        changed = True
                        break
                else:
                    # Общий случай: тип внутри скобок
                    tl = tl[len(prefix):-1].strip()
                    changed = True
                    break
    return tl

def needs_notnull_guard(t: str) -> bool:
    return t.strip().startswith('Nullable(')

def cond_for_type(col_expr: str, full_type: str) -> str:
    """
    Возвращает булево условие "значение задано/не по умолчанию" для столбца col_expr.
    """
    base = unwrap_type(full_type).lower()

    # Числа: Int*, UInt*, Float*, Decimal*
    if (base.startswith('int') or base.startswith('uint') or
        base.startswith('float') or base.startswith('decimal')):
        cond = f"{col_expr} != 0"
        if base.startswith('float'):
            # На всякий случай исключим NaN/Inf
            cond = f"({cond} AND isFinite({col_expr}) AND NOT isNaN({col_expr}))"
        return cond

    # Строки: String, FixedString(N)
    if base == 'string' or base.startswith('fixedstring'):
        return f"notEmpty({col_expr})"

    # LowCardinality(String) уже развёрнут выше до string, но на всякий случай:
    if base.startswith('lowcardinality('):
        return f"notEmpty({col_expr})"

    # Даты/время: Date, Date32, DateTime, DateTime64
    if base == 'date' or base == 'date32' or base.startswith('datetime'):
        # Приводим к UInt32: epoch-дни/секунды будут 0 для дефолта '1970-01-01'
        return f"toUInt32({col_expr}) != 0"

    # UUID
    if base == 'uuid':
        return f"{col_expr} != toUUID('00000000-0000-0000-0000-000000000000')"

    # IP
    if base == 'ipv4':
        return f"{col_expr} != toIPv4(0)"
    if base == 'ipv6':
        return f"{col_expr} != toIPv6('::')"

    # Enum
    if base.startswith('enum8') or base.startswith('enum16'):
        return f"toInt64({col_expr}) != 0"

    # Контейнеры: Array, Map, Tuple, Nested
    if base.startswith('array(') or base.startswith('map(') or base.startswith('tuple(') or base.startswith('nested('):
        return f"notEmpty({col_expr})"

    # Прочие сложные/экзотические типы (Object('JSON'), Variant и т.п.) — считаем «не NULL»
    return f"isNotNull({col_expr})"

# -----------------------------------


tables = ch.query_run('SHOW TABLES')['name'].to_list()

filtered_tables = [
    t for t in tables
    if (t.startswith('t_') 
        or 'maf_rep_mobile_total' in t
        or 'mobile_report_cum_rep_mobile_full' in t
        or 'mobile_report_rep_mobile_full' in t
        or 'mobile_report_total' in t
        or 'new_users_pd_rep_mobile_total' in t
        or 'rep_mobile_citizens_id_city_partner' in t
        or 'subscriptions_report_citizens_flats_comerce_rep_mobile_total' in t
        or 'subscriptions_report_comerce_rep_mobile_total' in t
        or 'total_active_users_per_day_full_table' in t
        or 'total_active_users_per_day_rep_mobile_total' in t
        or 'total_active_users_rep_mobile_total' in t
        or 'units_on_sk_platform_rep_mobile_total' in t
        ) 
        and not (t.endswith('_ch') 
                or t.endswith('_mv') 
                or t.endswith('_df') 
                or t.endswith('_mv_cod')
                or 't_bitrix_deals_sales' in t
                )
]

filtered_tables_mv_cod = [
    t for t in tables
    if t.endswith('_mv_cod')
]

for table in filtered_tables:
    mv_table = f"{table}_mv_cod"
    if mv_table in filtered_tables_mv_cod:
        query_text = f"""
        SYSTEM REFRESH VIEW {mv_table}
        """
        ch.query_run(query_text)
    else:
        cols = ch.get_schema_dict(f'SELECT * FROM {table} LIMIT 1')  # список кортежей (name, type)
        conds = []
        for col, ctype in cols:
            if col == 'report_date':
                continue
            col_expr = f"t.`{col}` = 0"
            conds.append(col_expr)

        where_clause = ' OR '.join(conds)

        group_cols = []
        for col, ctype in cols:
            col_quoted = f"`{col}`"

            if col == 'report_date':
                group_cols.append(col_quoted)
                continue

            base_condition = cond_for_type(col_quoted, ctype)

            # Если столбец Nullable — добавим явную проверку на не-NULL
            if needs_notnull_guard(ctype):
                condition = f"(isNotNull({col_quoted}) AND {base_condition})"
            else:
                condition = base_condition

            group_cols.append(f"countIf({condition}) AS `{col}`")

        select_clause = ', '.join(group_cols)

        query_text = f"""
        CREATE MATERIALIZED VIEW {mv_table}
        REFRESH EVERY 1 DAY OFFSET 4 HOUR 30 MINUTE RANDOMIZE FOR 10 MINUTE
        ENGINE = AggregatingMergeTree()
        ORDER BY report_date
        AS
        SELECT
            d.date AS report_date,
            t.* EXCEPT report_date
        FROM db1.date_range_mv AS d
        LEFT JOIN (
            SELECT
                {select_clause}
            FROM {table}
            GROUP BY report_date
        ) AS t
            ON d.date = t.report_date
        ORDER BY d.date DESC
        SETTINGS input_format_parallel_parsing = 0,
            date_time_input_format = 'best_effort'
        """

        print(query_text)
        print(mv_table)
        ch.query_run(query_text)

## Графики всех таблиц

In [ ]:
# pip install plotly pandas
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

DATE_COL = "report_date"
LABEL_FONT_SIZE = 14
LABEL_XSHIFT = -6

tables = ch.query_run("SHOW TABLES")["name"].to_list()
tables = [t for t in tables if t.endswith("_mv_cod")]

for table in tables:
    schema = ch.get_schema_dict(f"SELECT * FROM {table} LIMIT 1")
    cols = [name for name, _ in schema]
    if DATE_COL not in cols:
        continue
    y_cols = [c for c in cols if c != DATE_COL]
    if not y_cols:
        continue

    select_cols = ", ".join(f"`{c}`" for c in [DATE_COL] + y_cols)
    df = ch.query_run(f"SELECT {select_cols} FROM {table} ORDER BY {DATE_COL}")
    if df.empty:
        continue
    if df[DATE_COL].dtype == object:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")

    rows = len(y_cols)
    fig = make_subplots(rows=rows, cols=1, shared_xaxes=True, vertical_spacing=0.0)

    for i, col in enumerate(y_cols, start=1):
        # линия
        fig.add_trace(go.Scatter(x=df[DATE_COL], y=df[col], mode="lines", name=col),
                      row=i, col=1)

        # подпись у первой непустой точки
        s = df[[DATE_COL, col]].dropna()
        if not s.empty:
            fig.add_annotation(x=s[DATE_COL].iloc[0], y=s[col].iloc[0],
                               xref=f"x{i}", yref=f"y{i}",
                               text=col, showarrow=False,
                               xanchor="right", xshift=LABEL_XSHIFT,
                               font=dict(size=LABEL_FONT_SIZE))

        # <<< нулевая линия
        fig.add_shape(type="line",
                      xref=f"x{i}", yref=f"y{i}",
                      x0=df[DATE_COL].min(), x1=df[DATE_COL].max(),
                      y0=0, y1=0,
                      line=dict(color="black", width=3),
                      layer="below")

        # <<< гарантируем, что 0 попадёт в диапазон
        fig.update_yaxes(rangemode="tozero", row=i, col=1)

    fig.update_layout(
        title=dict(text=table, font=dict(size=28)),
        height=max(280, 95 * rows),
        showlegend=False,
        margin=dict(l=50, r=30, t=60, b=30)
    )
    fig.show()


In [ ]:
# pip install clickhouse-connect
import clickhouse_connect

def build_empty_sql(db, table, date_col='report_date', wide=False):
    cols = ch.query_run(f"""
        SELECT name, type
        FROM system.columns
        WHERE database = %(db)s AND table = %(table)s
        ORDER BY position
    """, parameters={'db': db, 'table': table}).result_rows

    def skip(t: str) -> bool:
      t = t.lower()
      return t.startswith(('array(', 'map(', 'aggregatefunction('))

    def empty_expr(col, t):
        colq, tlo = f'`{col}`', t.lower()

        def base(x):
            if 'string' in tlo:                          return f"{x} = ''"
            if tlo.startswith('date(') or tlo == 'date': return f"{x} = toDate(0)"
            if 'datetime' in tlo:                        return f"{x} = toDateTime(0)"
            if 'float' in tlo:                           return f"(({x} = 0) OR isNaN({x}))"
            if any(k in tlo for k in ('int', 'uint','decimal')): return f"{x} = 0"
            return "0"

        if 'nullable(' in tlo:
            return f"(isNull({colq}) OR {base(f'assumeNotNull({colq})')})"
        return base(colq)

    targets = [(c, t) for c, t in cols if c != date_col and not skip(t)]
    if not targets:
        raise ValueError("Нет подходящих колонок для подсчёта.")

    if not wide:
        keys = ", ".join(f"'{c}'" for c, _ in targets)
        vals = ", ".join(f"({empty_expr(c, t)})::UInt64" for c, t in targets)
        return f"""
/* tidy: report_date | column | empty_count */
SELECT
  {date_col} AS report_date,
  arrayJoin(keys)   AS column,
  arrayJoin(values) AS empty_count
FROM (
  SELECT {date_col}, sumMap(keys, vals) AS (keys, values)
  FROM (
    SELECT
      {date_col},
      [{keys}] AS keys,
      [{vals}] AS vals
    FROM `{db}`.`{table}`
  )
  GROUP BY {date_col}
)
ORDER BY report_date, column
""".strip()

    # wide
    aggs = ",\n  ".join(
        f"sum(({empty_expr(c, t)})::UInt64) AS empty_{c}" for c, t in targets
    )
    return f"""
/* wide: report_date + empty_<column>... */
SELECT
  {date_col},
  {aggs}
FROM `{db}`.`{table}`
GROUP BY {date_col}
ORDER BY {date_col}
""".strip()


In [ ]:
build_empty_counts_wide_sql('db1','cameras_dir_asgard_ch')

____

In [ ]:
def insert_new_date(table, special=False):
    query_text = f"""
        SELECT date_range_mv.report_date AS report_date
        FROM (SELECT DISTINCT date AS report_date FROM `date_range_mv`) AS date_range_mv
        LEFT ANTI JOIN
        (
            SELECT DISTINCT report_date
            FROM `{table}`
        ) ch USING (report_date)
        ORDER BY date_range_mv.report_date
    """

    df = ch.query_run(query_text)      # df: pandas.DataFrame
    missing = df['report_date'].tolist() if not df.empty else []
    dates_pd = pd.DataFrame({
            'report_date': [d.strftime('%Y-%m-%d') for d in missing]
            })
    dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
    for day_index in dates_pd.index:
        report_date = dates_pd.loc[day_index,['report_date']].values[0]
        query_text = open(f'{table}.txt','r').read().format(report_date)
        # приведём к литералу ClickHouse (подберите нужный тип)
        print(report_date)
        ch.query_run(query_text)
    
tables_list= ['total_active_users_per_day_full_table']
for table in tables_list:
    insert_new_date(table)


  report_date
0  2025-09-15
1  2025-09-14
2  2025-08-11
3  2024-08-04
4  2024-08-03
